## Demande de viz complémentaire axe 2.1 

En réponse au commentaire de Silvia du 21/0( sur doc étape 3 : "je verrai bien une carte avec les données Navigo de 1787, qui montre de quels ports français [pointcall OUT ou IN-OUT] sortent les navires de La Rochelle [homeport] et leur destination suivante avec une mise en valeur d'une manière ou d'une autre du tonnage (par ex: possibilité de cliquer pour voir le Nombre OU le tonnage total cumulé"

#### Données utilisées : 
pointcalls navigo : 
year = 1787
homeport = 'La Rochelle'
pointcall_action = ['Out', 'In-Out', 'In-out'] 

#### Construction de la carte : 
Production d'un jeu de données mettant en correspondance les points de départ ou d'étape de ces navires (attribut 'pointcall' des pointcalls 'Out' et 'In-Out'), avec leur destination (attribut 'pointcall' des pointcalls 'In' ayant le même source_doc_id).

Si le même trajet est réalisé plusieurs fois, on aggrège par nombre de trajets, ou tonnage cumulé, et on adapte la largeur des flux sur la carte.

In [ ]:
# imports
import json 
from poitousprint import Portic, get_pointcalls_commodity_purposes_as_toflit_product

portic_client = Portic()

In [ ]:
# collecte des départs et points d'étape
pointcalls_departures_la_rochelle_homeport = portic_client.get_pointcalls(
    year = 1787, 
    homeport = 'La Rochelle', # après vérification, tous les pointcalls 'Out' de 1789 qui concernent les ports rattachés au bureau des Fermes de Marennes ont bien l'attribut ferme_bureau à 'Marennes'
    pointcall_action = ['Out', 'In-Out', 'In-out'] # peut être encore des soucis de casse dans l'API 
)

print("nombre de pointcalls Out ou In-Out (départs et étapes) trouvés pour desnavires du port de La Rochelle:", len(pointcalls_departures_la_rochelle_homeport))

In [ ]:
# collecte de leurs source_doc_id
pointcalls_departures_source_doc_ids = set()

for p in pointcalls_departures_la_rochelle_homeport:
    pointcalls_departures_source_doc_ids.add(p['source_doc_id'])
    
print("Nombre de source_doc_id trouvés : ", len(pointcalls_departures_source_doc_ids))

In [ ]:
# retrouver les destinations : 1ere étape de collecte des pointcalls In
pointcalls_in_la_rochelle_homeport = portic_client.get_pointcalls(
    start_year = 1787,
    end_year = 1788, # on considère que le voyage ne dure pas plus d'un an
    pointcall_action = ['In', 'in'] # peut être encore des soucis de casse dans l'API 
)

print("Nombre de pointcalls In trouvés entre 1787 et 1788 :", len(pointcalls_in_la_rochelle_homeport))

In [ ]:
# retrouver les destinations : 2 etape de filtrage des pointcalls In considérés comme destination des Out sortant de Marennes avec du sel (même pkid)
pointcalls_destinations_la_rochelle_homeport = []

for p in pointcalls_in_la_rochelle_homeport:
    if (str(p['source_doc_id']) in pointcalls_departures_source_doc_ids):
        pointcalls_destinations_la_rochelle_homeport.append(p)

print("Nombre de destinations pertinentes trouvées : ", len(pointcalls_destinations_la_rochelle_homeport))

In [ ]:
import pandas as pds

# Load an empty map
from keplergl import KeplerGl

kepler_base_conf = {
    'version': 'v1',
    'config': {
        # centering the map on the region
        'mapState': {
            'latitude': 45.6876849,
            'longitude': -1.15,
            'zoom': 5.2
        }
    }
}

In [ ]:
flows_la_rochelle_homeport_1787 = {}
## but de reconstruire dicts qui mettent en relation les trajets qui ont le même source doc id mais je ne sais comment gérer les etapes
#{
#    'source_doc_id':
#    { 'lat_dep':
#      'lon_dep':
#      'lat_step': # après vérification on a au max 1 escale dans les données qu'on utilise
#      'lon_step': 
#      'lat_dest':
#      'lon_dest':
#      'tonnage':
#    }
#}

for p in pointcalls_departures_la_rochelle_homeport:
    port = p['pointcall']
    tonnage = int(p['tonnage']) if p['tonnage'] is not None else 0
    lat = float(p['latitude']) if p['latitude'] is not None else None
    lon = float(p['longitude']) if p['longitude'] is not None else None
    
    if p['source_doc_id'] not in flows_la_rochelle_homeport_1787:
        if p['pointcall_action']=='Out':
            flows_la_rochelle_homeport_1787[p['source_doc_id']] = {
                'lat_dep':lat,
                'lon_dep':lon,
                'port_dep':port,
                'tonnage':tonnage
            }
        else:
            flows_la_rochelle_homeport_1787[p['source_doc_id']] = {
                'lat_step':lat,
                'lon_step':lon,
                'port_step':port,
                'tonnage':tonnage
            }
    else:
        if p['pointcall_action']=='Out':
            flows_la_rochelle_homeport_1787[p['source_doc_id']]['port_dep'] = port
            flows_la_rochelle_homeport_1787[p['source_doc_id']]['lat_dep'] = lat
            flows_la_rochelle_homeport_1787[p['source_doc_id']]['lon_dep'] = lon
        else:
            flows_la_rochelle_homeport_1787[p['source_doc_id']]['port_step'] = port
            flows_la_rochelle_homeport_1787[p['source_doc_id']]['lat_step'] = lat
            flows_la_rochelle_homeport_1787[p['source_doc_id']]['lon_step'] = lon

for p in pointcalls_destinations_la_rochelle_homeport:
    port = p['pointcall']
    tonnage = int(p['tonnage']) if p['tonnage'] is not None else 0
    lat = float(p['latitude']) if p['latitude'] is not None else None
    lon = float(p['longitude']) if p['longitude'] is not None else None
    
    if p['source_doc_id'] not in flows_la_rochelle_homeport_1787:
        print("soucis") # on ne devrait pas avoir de source_doc_id inexistant
    else:
        flows_la_rochelle_homeport_1787[p['source_doc_id']]['port_dest'] = port
        flows_la_rochelle_homeport_1787[p['source_doc_id']]['lat_dest'] = lat
        flows_la_rochelle_homeport_1787[p['source_doc_id']]['lon_dest'] = lon

for key, values in flows_la_rochelle_homeport_1787.items():
    if 'lat_step' not in values:
        flows_la_rochelle_homeport_1787[key]['port_step'] = None
        flows_la_rochelle_homeport_1787[key]['lat_step'] = None
        flows_la_rochelle_homeport_1787[key]['lon_step'] = None
    
# print(flows_la_rochelle_homeport_1787)

unique_flows = {} # but de pouvoir cumuler le nombre de flows / le tonnage sur un même trajet (même trajet si même départ et même destination)
# un trajet unique est identifié par 'lat_dep_lon_dep_lat_dest_lon_dest'
for source_doc_id, values in flows_la_rochelle_homeport_1787.items():
    
    # j'ai 3 flows avec Err dans les coords de destination, et un 4e flow avec ces coords à None => a solutionner si temps
    
    if ('lat_dep' in values and 'lon_dep' in values and 'lat_dest' in values and 'lon_dest' in values):
        if str(values['lat_dep'])+'_'+str(values['lon_dep'])+'_'+str(values['lat_dest'])+'_'+str(values['lon_dest']) not in unique_flows:
            unique_flows[str(values['lat_dep'])+'_'+str(values['lon_dep'])+'_'+str(values['lat_dest'])+'_'+str(values['lon_dest'])] = {
                'port_dep':values['port_dep'],
                'port_dest':values['port_dest'],
                'lat_dep':values['lat_dep'],
                'lon_dep':values['lon_dep'],
                'lat_dest':values['lat_dest'],
                'lon_dest':values['lon_dest'],
                'nb_flows': 1,
                'tonnages_cumulés':values['tonnage']
            }
        else:
            unique_flows[str(values['lat_dep'])+'_'+str(values['lon_dep'])+'_'+str(values['lat_dest'])+'_'+str(values['lon_dest'])]['nb_flows'] += 1
            unique_flows[str(values['lat_dep'])+'_'+str(values['lon_dep'])+'_'+str(values['lat_dest'])+'_'+str(values['lon_dest'])]['tonnages_cumulés'] += values['tonnage']

flows_la_rochelle_homeport_1787 = [{'source_doc_id': source_doc_id, **vals} for source_doc_id, vals in flows_la_rochelle_homeport_1787.items()]
unique_flows = [{'flow_id': flow_id, **vals} for flow_id, vals in unique_flows.items()]

with open('assets/all_flows.json', 'w') as fp1:
    json.dump(flows_la_rochelle_homeport_1787, fp1)

with open('assets/aggregated_flows.json', 'w') as fp2:
    json.dump(unique_flows, fp2)


In [ ]:
# carte finale sur laquelle par défaut on arrive sur la visualisation des tonnages cumulés avec départs en violet / arrivées en orange
# autre layer pertinent : "nombre de trajets" avec départs en bleu et arrivées en jaune
# (les autres layers sont obsolètes) 

flows_la_rochelle_homeport_1787_map = KeplerGl(config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "3s6azhf",
          "type": "line",
          "config": {
            "dataId": "lgvfx8vid",
            "label": "nombre de trajets cumulés ",
            "color": [
              71,
              211,
              217
            ],
            "columns": {
              "lat0": "lat_dep",
              "lng0": "lon_dep",
              "lat1": "lat_dest",
              "lng1": "lon_dest",
              "alt0": None,
              "alt1": None
            },
            "isVisible": False,
            "visConfig": {
              "opacity": 0.8,
              "thickness": 2,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "sizeRange": [
                1,
                50
              ],
              "targetColor": [
                237,
                200,
                0
              ],
              "elevationScale": 1
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "sizeField": {
              "name": "nb_flows",
              "type": "integer"
            },
            "sizeScale": "linear"
          }
        },
        {
          "id": "vz7qk6q",
          "type": "line",
          "config": {
            "dataId": "lgvfx8vid",
            "label": "tonnages cumulés",
            "color": [
              122,
              13,
              166
            ],
            "columns": {
              "lat0": "lat_dep",
              "lng0": "lon_dep",
              "lat1": "lat_dest",
              "lng1": "lon_dest",
              "alt0": None,
              "alt1": None
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.8,
              "thickness": 2,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "sizeRange": [
                1,
                50
              ],
              "targetColor": [
                250,
                116,
                0
              ],
              "elevationScale": 1
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "sizeField": {
              "name": "tonnages_cumulés",
              "type": "integer"
            },
            "sizeScale": "linear"
          }
        },
        {
          "id": "z57i6tm",
          "type": "line",
          "config": {
            "dataId": "q5if54fm9",
            "label": "trajets avec duplication",
            "color": [
              23,
              184,
              190
            ],
            "columns": {
              "lat0": "lat_dep",
              "lng0": "lon_dep",
              "lat1": "lat_dest",
              "lng1": "lon_dest",
              "alt0": None,
              "alt1": None
            },
            "isVisible": False,
            "visConfig": {
              "opacity": 0.8,
              "thickness": 2,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "sizeRange": [
                1,
                10
              ],
              "targetColor": None,
              "elevationScale": 1
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "sizeField": {
              "name": "tonnage",
              "type": "integer"
            },
            "sizeScale": "linear"
          }
        },
        {
          "id": "4fwtupi",
          "type": "arc",
          "config": {
            "dataId": "q5if54fm9",
            "label": "Out to In-Out",
            "color": [
              122,
              13,
              166
            ],
            "columns": {
              "lat0": "lat_dep",
              "lng0": "lon_dep",
              "lat1": "lat_step",
              "lng1": "lon_step"
            },
            "isVisible": False,
            "visConfig": {
              "opacity": 0.8,
              "thickness": 2,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "sizeRange": [
                1,
                29.4
              ],
              "targetColor": [
                130,
                13,
                175
              ]
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "sizeField": {
              "name": "tonnage",
              "type": "integer"
            },
            "sizeScale": "linear"
          }
        },
        {
          "id": "k51tdn",
          "type": "arc",
          "config": {
            "dataId": "q5if54fm9",
            "label": "In-Out to In",
            "color": [
              253,
              236,
              0
            ],
            "columns": {
              "lat0": "lat_step",
              "lng0": "lon_step",
              "lat1": "lat_dest",
              "lng1": "lon_dest"
            },
            "isVisible": False,
            "visConfig": {
              "opacity": 0.8,
              "thickness": 2,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "sizeRange": [
                0,
                200
              ],
              "targetColor": [
                255,
                248,
                51
              ]
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "sizeField": {
              "name": "tonnage",
              "type": "integer"
            },
            "sizeScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "q5if54fm9": [
              {
                "name": "source_doc_id",
                "format": None
              },
              {
                "name": "tonnage",
                "format": None
              }
            ],
            "lgvfx8vid": [
              {
                "name": "flow_id",
                "format": None
              },
              {
                "name": "port_dep",
                "format": None
              },
              {
                "name": "port_dest",
                "format": None
              },
              {
                "name": "nb_flows",
                "format": None
              },
              {
                "name": "tonnages_cumulés",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 23.539800880918328,
      "longitude": -41.37482716434267,
      "pitch": 0,
      "zoom": 1.7251439219629985,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}, data={
    'q5if54fm9': pds.DataFrame(flows_la_rochelle_homeport_1787),
    'lgvfx8vid': pds.DataFrame(unique_flows)
    })

flows_la_rochelle_homeport_1787_map